(hover_dir_pad_1, hover_dir_pad_2, hover_numeric_pad)

(5 options, 5 options, 11 options) ~ 275

5 possible actions 

total 1375 state, action combinations

initial state is (A, A, A)
let's say we have to type 456A

(A, A, A)
(A, A, 4) A
(A, A, 5) A
(A, A, 6) A
(A, A, A) A


what if we're in like

(<, ^, 5) and we hit >
(A, ^, 5) and we hit >


(<, ^, 5) and we hit A?

(A, ^, 5) and we hit A?
(^, A, 5) and we hit A?
(^, >, A) and we hit A?

(A, A, 5) and we hit A?
(^, A, A) and we hit A?

(A, A, A) and we hit A?

In [1]:
# input = """029A
# 980A
# 179A
# 456A
# 379A"""

input = open("inputs/21").read()

parsed_input = [list(s) for s in input.split()]

In [2]:
parsed_input

[['1', '4', '0', 'A'],
 ['1', '7', '0', 'A'],
 ['1', '6', '9', 'A'],
 ['8', '0', '3', 'A'],
 ['1', '2', '9', 'A']]

In [3]:
keypresses = parsed_input[0]
keypresses

['1', '4', '0', 'A']

In [16]:
import numpy as np

directional_board = np.array(
    [
        [None, "^", "A"],
        ["<", "v", ">"],
    ]
)

numeric_board = np.array(
    [
        ["7", "8", "9"],
        ["4", "5", "6"],
        ["1", "2", "3"],
        [None, "0", "A"],
    ]
)

dirs = {
    "^": (-1, 0),
    ">": (0, 1),
    "v": (1, 0),
    "<": (0, -1),
}


def in_bounds(tuple, shape):
    return 0 <= tuple[0] < shape[0] and 0 <= tuple[1] < shape[1]


def add_tuple(t1, t2):
    return tuple(map(sum, zip(t1, t2)))


def make_transition_dict(board):
    d = {}

    for i, j in np.ndindex(board.shape):
        if board[i, j] is None:
            continue

        for dir in dirs:
            new_pos = add_tuple((i, j), dirs[dir])
            d[(board[i, j], dir)] = (
                board[new_pos] if in_bounds(new_pos, board.shape) else None
            )

    return d


numeric_transition_dict = make_transition_dict(numeric_board)
directional_transition_dict = make_transition_dict(directional_board)

In [5]:
def get_next_state(state, action):
    dir_hover_1, dir_hover_2, num_hover = state

    # easy case. we just trigger movements in dir 1 if feasible
    if action != "A":
        # print("easy case")
        new_dir_hover_1 = directional_transition_dict[dir_hover_1, action]
        if not new_dir_hover_1:
            return None

        return (new_dir_hover_1, dir_hover_2, num_hover), None

    # more complex if action is A
    # cases reduce to how many consecutive A states we're in, I think
    if dir_hover_1 == "A" and dir_hover_2 == "A":
        # print("double hover A")
        # we emit whatever num_hover is
        # otherwise no change
        # print(f"Emit {num_hover}")
        return state, num_hover
    elif dir_hover_1 == "A" and dir_hover_2 != "A":
        # print("first hover A")
        new_num_hover = numeric_transition_dict[num_hover, dir_hover_2]
        if not new_num_hover:
            return None

        return (dir_hover_1, dir_hover_2, new_num_hover), None
    else:
        # print("first not hover A")
        new_dir_hover_2 = directional_transition_dict[dir_hover_2, dir_hover_1]
        if not new_dir_hover_2:
            return None
        return (dir_hover_1, new_dir_hover_2, num_hover), None

In [6]:
# sequence = "<vA<AA>>^AvAA<^A>A<v<A>>^AvA^A<vA>^A<v<A>^A>AAvA^A<v<A>A>^AAAvA<^A>A"
# sequence = "<v<A>>^AAAvA^A<vA<AA>>^AvAA<^A>A<v<A>A>^AAAvA<^A>A<vA>^A<A>A"
# sequence = "<v<A>>^A<vA<A>>^AAvAA<^A>A<v<A>>^AAvA^A<vA>^AA<A>A<v<A>A>^AAAvA<^A>A"
# sequence = "<v<A>>^AA<vA<A>>^AAvAA<^A>A<vA>^A<A>A<vA>^A<A>A<v<A>A>^AAvA<^A>A"
sequence = "<v<A>>^AvA^A<vA<AA>>^AAvA<^A>AAvA^A<vA>^AA<A>A<v<A>A>^AAAvA<^A>A"

In [7]:
state = ("A", "A", "A")

for action in sequence:
    if result := get_next_state(state, action):
        state, emit = result
        if emit:
            print(emit)

3
7
9
A


In [8]:
def get_possible_neighbors(state):
    for action in list(dirs.keys()) + ["A"]:
        if result := get_next_state(state, action):
            # state, emit = result
            yield result

In [9]:
list(get_possible_neighbors(("A", "A", "A")))

[(('>', 'A', 'A'), None), (('^', 'A', 'A'), None), (('A', 'A', 'A'), 'A')]

In [10]:
list(get_possible_neighbors(("A", "^", "A")))

[(('>', '^', 'A'), None), (('^', '^', 'A'), None), (('A', '^', '3'), None)]

In [11]:
# from collections import deque


# def get_path_length_between_states(start_state, end_state):
#     """bfs"""
#     to_visit = deque([(start_state, 0)])
#     visited = set()

#     while to_visit:
#         current_node, distance = to_visit.popleft()

#         if current_node == end_state:
#             return distance

#         if current_node in visited:
#             continue

#         for n, _emit in get_possible_neighbors(current_node):
#             to_visit.append((n, distance + 1))

#         visited.add(current_node)

#     return None


In [12]:
get_path_length_between_states(("A", "A", "A"), ("A", "A", "9"))

13

In [13]:
s = 0

for code in parsed_input:
    numeric_part = int("".join(code[:-1]))
    to_reach = [("A", "A", "A")] + [("A", "A", c) for c in code]

    length = 0
    for state_1, state_2 in zip(to_reach, to_reach[1:]):
        d = get_path_length_between_states(state_1, state_2)
        length += d + 1  # plus 1 for the extra 'A' action needed to emit
        # print(state_1, state_2)
        # print(d)

    s += length * numeric_part

s

105458